### Import libraries

In [83]:
import tensorflow as tf
import numpy as np
import os
import random


tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.10.0


### Set seed for reproducibility

In [84]:
# Random seed for reproducibility
seed = 41

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Suppress warnings

In [85]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [86]:
dataset_dir = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data'
training = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data/training'
validation = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data/validation'
test = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data/test'

In [87]:
labels = {
    0:'species1',
    1:'species2',
    2:'species3',
    3:'species4',
    4:'species5',
    5:'species6',
    6:'species7',
    7:'species8'
}

### Process data

In [88]:
# Images are divided into folders, one for each class.
# If the images are organized in such a way, we can exploit the
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input

# Create an instance of ImageDataGenerator for training, validation, and test sets
train_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
test_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=training,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed
                                               )
valid_gen = train_data_gen.flow_from_directory(directory=validation,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed
                                               )
test_gen = train_data_gen.flow_from_directory(directory=test,
                                              target_size=(96,96),
                                              color_mode='rgb',
                                              classes=None, # can be set to labels
                                              class_mode='categorical',
                                              batch_size=32,
                                              shuffle=False,
                                              seed=seed
                                              )

Found 2669 images belonging to 8 classes.
Found 381 images belonging to 8 classes.
Found 492 images belonging to 8 classes.


In [89]:
# Create an instance of ImageDataGenerator with Data Augmentation
aug_train_data_gen = ImageDataGenerator(rotation_range=30,
                                        horizontal_flip= True,
                                        vertical_flip= True,
                                        brightness_range=(0.6,1.4),
                                        zoom_range=0.6,
                                        fill_mode='nearest',
                                        preprocessing_function= preprocess_input) # rescale value is multiplied to the image

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=training,
                                                       target_size=(96,96),
                                                       color_mode='rgb',
                                                       classes=None, # can be set to labels
                                                       class_mode='categorical',
                                                       batch_size=32,
                                                       shuffle=True,
                                                       seed=seed)

Found 2669 images belonging to 8 classes.


### Models metadata

In [90]:
input_shape = (96, 96, 3)
batch_size = 32
epochs = 200

### Standard model

### Transfer learning

In [91]:
# Download and plot the VGG16 model
supernet = tfk.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(96,96,3)
)
supernet.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 47, 47, 32)   864         ['input_7[0][0]']                
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 47, 47, 32)  128         ['block1_conv1[0][0]']           
 ation)                                                                                           
                                                                                                  
 block1_conv1_act (Activation)  (None, 47, 47, 32)   0           ['block1_conv1_bn[0][0]'] 

In [92]:
# Use the supernet as feature extractor
supernet.trainable = False

inputs = tfk.Input(shape=(96,96,3))
x = supernet(inputs)
x = tfkl.Flatten(name='Flattening')(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
x = tfkl.Dense(
    128,
    activation='relu',
    kernel_initializer = tfk.initializers.HeUniform(seed))(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
outputs = tfkl.Dense(
    8,
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


# Connect input and output through the Model class
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
tl_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 xception (Functional)       (None, 3, 3, 2048)        20861480  
                                                                 
 Flattening (Flatten)        (None, 18432)             0         
                                                                 
 dropout_6 (Dropout)         (None, 18432)             0         
                                                                 
 dense_6 (Dense)             (None, 128)               2359424   
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 8)                 1032  

In [93]:
# Train the model
tl_history = tl_model.fit(
    x = aug_train_gen,
    batch_size = 32,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

Epoch 1/200
84/84 [==============================] - 20s 209ms/step - loss: 2.2168 - accuracy: 0.2237 - val_loss: 1.8835 - val_accuracy: 0.2992
Epoch 2/200
84/84 [==============================] - 14s 161ms/step - loss: 1.8783 - accuracy: 0.2997 - val_loss: 1.7556 - val_accuracy: 0.3596
Epoch 3/200
84/84 [==============================] - 14s 160ms/step - loss: 1.7458 - accuracy: 0.3477 - val_loss: 1.6124 - val_accuracy: 0.4199
Epoch 4/200
84/84 [==============================] - 15s 177ms/step - loss: 1.6934 - accuracy: 0.3574 - val_loss: 1.5958 - val_accuracy: 0.4357
Epoch 5/200
84/84 [==============================] - 15s 175ms/step - loss: 1.6716 - accuracy: 0.3750 - val_loss: 1.5885 - val_accuracy: 0.3937
Epoch 6/200
84/84 [==============================] - 14s 163ms/step - loss: 1.6547 - accuracy: 0.3897 - val_loss: 1.5296 - val_accuracy: 0.4541
Epoch 7/200
84/84 [==============================] - 13s 158ms/step - loss: 1.6059 - accuracy: 0.3900 - val_loss: 1.5259 - val_accuracy:

In [94]:
# Save the best model

tl_model.save('TransferLearningModel')
del tl_model

In [95]:
model_aug = tfk.models.load_model("TransferLearningModel")
model_aug_test_metrics = model_aug.evaluate(test_gen, return_dict=True)

print()
print("Test metrics with VGG")
print(model_aug_test_metrics)

16/16 [==============================] - 7s 287ms/step - loss: 1.3769 - accuracy: 0.5264

Test metrics with VGG
{'loss': 1.3769094944000244, 'accuracy': 0.5264227390289307}


### Fine tuning

In [96]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('TransferLearningModel')
ft_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 xception (Functional)       (None, 3, 3, 2048)        20861480  
                                                                 
 Flattening (Flatten)        (None, 18432)             0         
                                                                 
 dropout_6 (Dropout)         (None, 18432)             0         
                                                                 
 dense_6 (Dense)             (None, 128)               2359424   
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 8)                 1032  

In [97]:
# Set all VGG layers to True
ft_model.get_layer('xception').trainable = True
for i, layer in enumerate(ft_model.get_layer('xception').layers):
   print(i, layer.name, layer.trainable)

0 input_7 True
1 block1_conv1 True
2 block1_conv1_bn True
3 block1_conv1_act True
4 block1_conv2 True
5 block1_conv2_bn True
6 block1_conv2_act True
7 block2_sepconv1 True
8 block2_sepconv1_bn True
9 block2_sepconv2_act True
10 block2_sepconv2 True
11 block2_sepconv2_bn True
12 conv2d_12 True
13 block2_pool True
14 batch_normalization_12 True
15 add_36 True
16 block3_sepconv1_act True
17 block3_sepconv1 True
18 block3_sepconv1_bn True
19 block3_sepconv2_act True
20 block3_sepconv2 True
21 block3_sepconv2_bn True
22 conv2d_13 True
23 block3_pool True
24 batch_normalization_13 True
25 add_37 True
26 block4_sepconv1_act True
27 block4_sepconv1 True
28 block4_sepconv1_bn True
29 block4_sepconv2_act True
30 block4_sepconv2 True
31 block4_sepconv2_bn True
32 conv2d_14 True
33 block4_pool True
34 batch_normalization_14 True
35 add_38 True
36 block5_sepconv1_act True
37 block5_sepconv1 True
38 block5_sepconv1_bn True
39 block5_sepconv2_act True
40 block5_sepconv2 True
41 block5_sepconv2_bn Tru

In [98]:
# Freeze first N layers, e.g., until 14th
for i, layer in enumerate(ft_model.get_layer('xception').layers[:40]):
  layer.trainable=False
for i, layer in enumerate(ft_model.get_layer('xception').layers):
   print(i, layer.name, layer.trainable)
ft_model.summary()

0 input_7 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d_12 False
13 block2_pool False
14 batch_normalization_12 False
15 add_36 False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_13 False
23 block3_pool False
24 batch_normalization_13 False
25 add_37 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_14 False
33 block4_pool False
34 batch_normalization_14 False
35 add_38 False
36 block5_sepconv1_act False
37 block5_sepconv1 False
38 block5_sepconv1_bn False
39 block5_sepconv2_act False
40 block5

In [99]:
# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [100]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = aug_train_gen,
    batch_size = 32,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

Epoch 1/200
84/84 [==============================] - 50s 518ms/step - loss: 1.5872 - accuracy: 0.3945 - val_loss: 1.3871 - val_accuracy: 0.5039
Epoch 2/200
84/84 [==============================] - 30s 352ms/step - loss: 1.4004 - accuracy: 0.4713 - val_loss: 1.3006 - val_accuracy: 0.5617
Epoch 3/200
84/84 [==============================] - 29s 344ms/step - loss: 1.2930 - accuracy: 0.5159 - val_loss: 1.3146 - val_accuracy: 0.5328
Epoch 4/200
84/84 [==============================] - 27s 325ms/step - loss: 1.2286 - accuracy: 0.5451 - val_loss: 1.3184 - val_accuracy: 0.5039
Epoch 5/200
84/84 [==============================] - 27s 318ms/step - loss: 1.1571 - accuracy: 0.5766 - val_loss: 1.6492 - val_accuracy: 0.5302
Epoch 6/200
84/84 [==============================] - 27s 320ms/step - loss: 1.0987 - accuracy: 0.6107 - val_loss: 1.2906 - val_accuracy: 0.5381
Epoch 7/200
84/84 [==============================] - 27s 315ms/step - loss: 1.0484 - accuracy: 0.6208 - val_loss: 1.2388 - val_accuracy:

In [101]:
ft_model.save('FineTuningModel')
del ft_model

In [102]:
model_aug = tfk.models.load_model("FineTuningModel")
model_aug_test_metrics = model_aug.evaluate(test_gen, return_dict=True)

print()
print("Test metrics with VGG")
print(model_aug_test_metrics)

16/16 [==============================] - 7s 340ms/step - loss: 0.5961 - accuracy: 0.8293

Test metrics with VGG
{'loss': 0.5960544347763062, 'accuracy': 0.8292682766914368}
